<a href="https://colab.research.google.com/github/himanshunaidu/cnn_image_caption/blob/master/ImageCaption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS

In [1]:
from google.colab import files
uploaded = files.upload()

Saving load_dataset.py to load_dataset.py
Saving preprocess_images.py to preprocess_images.py
Saving preprocess_labels.py to preprocess_labels.py


In [28]:
import os
import tensorflow as tf
import numpy as np
import math
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2, inception_v3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as kb
from tqdm import tqdm
import re

In [2]:
print(tf.__version__)

2.5.0


In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
!ls

gdrive		 preprocess_images.py  __pycache__
load_dataset.py  preprocess_labels.py  sample_data


# LOCAL IMPORTS AND VARIABLES

In [5]:
from load_dataset import loadCaptions, getImages
from preprocess_images import load_image, get_images, preprocess_images
from preprocess_labels import captions_clean, add_token, subset_data_dict, all_captions, max_caption_length, create_tokenizer

In [6]:
base_path = './gdrive/MyDrive/ImageCaptions/'
caption_file_path = './Flickr8k.token.txt'
ds_path = 'Flicker8k_Dataset/'
feature_path = 'Flicker8k_Dataset_Features/'
filler_path = 'Filler/'
train_images_path = 'Flickr_8k.trainImages.txt'

# LOAD IMAGES AND CAPTIONS

In [7]:
img_list = getImages(os.path.join(base_path, train_images_path))

In [8]:
capdict = loadCaptions(os.path.join(base_path, caption_file_path))

In [ ]:
capdictset = {}
for i, key in enumerate(capdict.keys()):
  if i>3: break
  capdictset[key] = capdict.get(key)

# PREPROCESS IMAGES

In [9]:
img_name_list, img_path_list, img_dataset = get_images(base_path, train_images_path, ds_path)

6001
./gdrive/MyDrive/ImageCaptions/Flicker8k_Dataset/2513260012_03d33305cf.jpg


In [10]:
# USE THIS FUNCTION TO SAVE ALL THE PRE-PROCESSED IMAGE FEATURES AS GENERATED BY INCEPTIONV3
# preprocess_images(img_name_list, img_path_list, img_dataset, ds_path, feature_path)

# PREPROCESS CAPTIONS

In [ ]:
# USE THE FUNCTIONS TO PRE-PROCESS ALL THE CAPTIONS

In [14]:
captions_clean(capdict)

In [24]:
capdict[list(capdict.keys())[0]]
# list(capdict.keys())[0]

['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq girl going into wooden building endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl in pink dress going into wooden cabin endseq']

In [17]:
training_dict = subset_data_dict (capdict, img_name_list) #startseq and enseq are being added

In [ ]:
training_dict

In [20]:
tokenizer, vocab_size, max_caption_words = create_tokenizer(training_dict)

# PREP DATASET

In [31]:
word_idxs = tokenizer.texts_to_sequences(['startseq child in pink dress is climbing up set of stairs in an entry way endseq'])[0]
pad_idxs = pad_sequences([word_idxs], maxlen=max_caption_words, padding='post')[0]
len(pad_idxs)

36

In [34]:
def data_prep(data_dict, tokenizer, max_length, vocab_size):
  X, y = list(), list()

  # For each image and list of captions
  for image_name, captions in data_dict.items():
    image_name = os.path.join(base_path, ds_path, image_name)

    # For each caption in the list of captions
    for caption in captions:

      # Convert the caption words into a list of word indices
      word_idxs = tokenizer.texts_to_sequences([caption])[0]

      # Pad the input text to the same fixed length
      # pad_idxs = pad_text(word_idxs, max_length)
      pad_idxs = pad_sequences([word_idxs], maxlen=max_length, padding='post')[0]
          
      X.append(image_name)
      y.append(pad_idxs)
  
  return np.array(X), np.array(y)
  return X, y

In [35]:
train_X, train_y = data_prep(training_dict, tokenizer, max_caption_words, vocab_size)

In [36]:
print(train_X[0], train_y[0])

./gdrive/MyDrive/ImageCaptions/Flicker8k_Dataset/1000268201_693b08cb0e.jpg [   1   42    3   87  170    6  116   52  387   11  394    3   27 4436
  626    2    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
